In [ ]:
%pylab inline

# Loading data from a text file

The following cell loads the data stored in the text files "train.txt" and "test.txt". This results in two NumPy arrays with shapes 500x5 (train.txt) and 5000x5 (test.txt) - 500 and 5000 samples of the following 5 random variables:

Column 0: S ... stress (false (0) or true (1))     
Column 1: E ... easily catches cold (false (0) or true (1))  
Column 2: G ... genetic disposition (false (0) or true (1))   
Column 3: I ... increased blood pressure (false (0) or true (1))   
Column 4: H ... heart attack (false (0) or true (1))   

In [28]:
# load the training dataset Y. Note: the file "train.txt" has to be in the same directory you started the ipython notebook server in
Y = np.loadtxt('train.txt', dtype=int)

# load the training dataset Z (used for last exercise). Again, the file has to be in correct directory.
Z = np.loadtxt('test.txt', dtype=int)

# row indices of the random variables
_s_, _e_, _g_, _i_, _h_ = 0, 1, 2, 3, 4

na = newaxis

# Helper functions for probability tables

In [29]:
def mle_1d(X, x):
    """ 
    Probability distribution of a variable: P(VAR)
    :param X: matrix array of boolean variables, in an integer shape (zeroes and ones)
    :param x: column of the X matrix of which we want the probability distribution
    :returns: array with the probability distribution of the required variable, given the array
    """
    sm = X[:, x].sum()
    num = X[:, x].shape[0] 
    return array([1 - sm / float(num), sm / float(num)])
    
def mle_2d(X, a, b):
    """ 
    Conditional probability distribution of a variable, given another variable: P(VAR1 | VAR2)
    :param X: matrix array of boolean variables, in an integer shape (zeroes and ones)
    :param a: column of the X matrix of which we want the conditional probability distribution
    :param b: conditioning column of the X matrix
    :returns: array with the probability distribution of the required variable, given the array and the conditioning variable
    """
    x = X[:, a]
    y = X[:, b]
    num = X.shape[0] 
    condA = where((x == 0) & (y == 0))[0].shape[0] / float(num)
    condB = where((x == 0) & (y == 1))[0].shape[0] / float(num)
    condC = where((x == 1) & (y == 0))[0].shape[0] / float(num)
    condD = 1 - (condA + condB + condC)   
    ab = array([[condA, condB], [condC, condD]])
    return ab / ab.sum(1) / (ab / ab.sum(1)).sum(0)
    

def mle_3d(X, a, b, c):  
    """ 
    Conditional probability distribution of a variable, given two other variables: P(VAR1 | VAR2, VAR3)
    :param X: matrix array of boolean variables, in an integer shape (zeroes and ones)
    :param a: column of the X matrix of which we want the conditional probability distribution
    :param b: first conditioning column of the X matrix
    :param c: second conditioning column of the X matrix
    :returns: array with the probability distribution of the required variable, given the array and the conditioning variables
    """    
    x = X[:, a]
    y = X[:, b]
    z = X[:, c]
    num = X.shape[0] 
    condA = where((x == 0) & (y == 0) & (z == 0))[0].shape[0] / float(num)
    condB = where((x == 0) & (y == 0) & (z == 1))[0].shape[0] / float(num)
    condC = where((x == 0) & (y == 1) & (z == 0))[0].shape[0] / float(num)
    condD = where((x == 0) & (y == 1) & (z == 1))[0].shape[0] / float(num)
    condE = where((x == 1) & (y == 0) & (z == 0))[0].shape[0] / float(num)
    condF = where((x == 1) & (y == 0) & (z == 1))[0].shape[0] / float(num)
    condG = where((x == 1) & (y == 1) & (z == 0))[0].shape[0] / float(num)
    condH = 1 - (condA + condB + condC + condD + condE + condF + condG)  
    abc =  array([[[condA, condB], [condC, condD]], [[condE, condF], [condG, condH]]])   
    bc = abc.sum(0)
    return abc / bc

# 1.1.A) Probability tables
of P(S), P(G), P(I), P(E), P(H).

In [30]:
S = mle_1d(Y,_s_)
G = mle_1d(Y,_g_)
I = mle_1d(Y,_i_)
E = mle_1d(Y,_e_)
H = mle_1d(Y,_h_)

# 1.1.B) log likelihood of model 1 relative to the train dataset.

In [32]:
SEGIH1 = S[:, na, na, na, na] * E[na, :, na, na, na] * G[na, na, :, na, na] * I[na, na, na, :, na] * H[na, na, na, na, :] 
ll1r = log(SEGIH1[Y[:,0],Y[:,1],Y[:,2],Y[:,3],Y[:,4]]).sum()
print 'The log likelihood of model 1 relative to the train dataset is ' + `ll1r` + ' .'

The log likelihood of model 1 relative to the train dataset is -970.81190292576616 .


# 1.2.A) Probability tables
of P(E|S), P (I | G, S), and P (H | I) (The remaining ones are already given by 1.1.A)).

In [33]:
E_S = mle_2d(Y, _e_, _s_)
H_I = mle_2d(Y, _h_, _i_)
I_GS = mle_3d(Y, _i_, _g_, _s_)

# 1.2.B) log likelihood of model 2 relative to the train dataset

In [34]:
SEGIH2 = S[:, na, na, na, na] * E_S.transpose((1, 0))[:, :, na, na, na] * G[na, na, :, na, na] * I_GS.transpose((2, 1, 0))[:, na, :, :, na] * H_I.transpose((1, 0))[na, na, na, :, :] 
ll2r = log(SEGIH2[Y[:,0],Y[:,1],Y[:,2],Y[:,3],Y[:,4]]).sum()
print 'The log likelihood of model 2 relative to the train dataset is ' + `ll2r` + ' .'

The log likelihood of model 2 relative to the train dataset is -940.76297860888167 .


# 1.3.A) Probability tables
of P (S | G), P (E | S, I), and P (H | I, E) (The remaining ones are already given by 1.1.A) and 1.2.A)).


In [2]:
S_G = mle_2d(Y, _s_, _g_)
E_SI = mle_3d(Y, _e_, _s_, _i_)
H_IE = mle_3d(Y, _h_, _i_, _e_)

NameError: name 'mle_2d' is not defined

# 1.3.B) log likelihood of model 3 relative to the train dataset

In [1]:
SEGIH3 = S_G[:, na, :, na, na] * E_SI.transpose((1, 0, 2))[:, :, na, :, na] * G[na, na, :, na, na] * I_GS.transpose((2, 1, 0))[:, na, :, :, na] * H_IE.transpose((2, 1, 0))[na, :, na, :, :] 
ll3r = log(SEGIH3[Y[:,0],Y[:,1],Y[:,2],Y[:,3],Y[:,4]]).sum()
print 'The log likelihood of model 3 relative to the train dataset is ' + `ll3r` + ' .'

NameError: name 'S_G' is not defined

# 2.A) Compare training data
We can state that minimizing the negative log likelihood is equivalent to maximize the likelihood estimation (MLE).  
The model that maximizes the likelihood is clearly the best.

In [ ]:
trainll = abs(array([ll1r, ll2r, ll3r]))
argmin(trainll)
min(trainll)
print 'The best model for the training data is ' + `argmin(trainll) + 1` + ', '
print 'because is the smallest, with a negative log likelihood of ' + `min(trainll)` + ' .'

The third model, the most complex of the three models, with a small training set, looks like to be the most effective.  
The second model has a log likelihood very close to the third, leaving us wondering if all that complexity was really necessary.  
The first model is the worst by far, probably due to ist excessive simplicity.

# 2.B) Compare test data

In [43]:
testll = abs(array([log(SEGIH1[Z[:,0],Z[:,1],Z[:,2],Z[:,3],Z[:,4]]).sum(), 
                    log(SEGIH2[Z[:,0],Z[:,1],Z[:,2],Z[:,3],Z[:,4]]).sum(),
                    log(SEGIH3[Z[:,0],Z[:,1],Z[:,2],Z[:,3],Z[:,4]]).sum()]))
print testll
argmin(testll)
min(testll)
print 'The best model for the test data is ' + `argmin(testll) + 1` + ', '
print 'because is the smallest, with a negative log likelihood of ' + `min(testll)` + ' .'

[ 9545.45920494  9216.89949044  9226.91241436]
The best model for the test data is 2, 
because is the smallest, with a negative log likelihood of 9216.89949044479 .


The second model, the most reasonable of the three models, with a large test set, is the most effective.  
The third model, as feared, performs worse than the second, probably because of overfitting.  
The first model is again the worst by far, again probably due to ist excessive simplicity.

*Given the reasons in 3.A) and 3.B), I would choose the second model as the best for heart diseases.*